Take the input from file 'data/generated_recipes.json' and check each recipe for correctness.

The RAG model first takes the name of the medicine from field 'medicijn' and retrieves the documentation from apotheek.nl. The pattern for the URL is:
https://www.apotheek.nl/medicijnen/{medicijn}

Then it checks if the medicine is valid by checking if the URL returns a 200 status code. If the medicine can not be found, the recipe is marked as invalid.

If the medicine is valid the documentation is provided as context to the AI model to check if the strength, form, number of doses, and instructions are valid. The AI model is also asked to check if the indication is relevant to the medical question.


In [19]:
from bs4 import BeautifulSoup
import unicodedata
import requests

def get_medicijn_details(medicijn: str):
    normalized_medicijn = unicodedata.normalize('NFKD', medicijn.lower())
    normalized_medicijn = ''.join(c for c in normalized_medicijn if not unicodedata.combining(c))

    """Retrieve details of the medicine from apotheek.nl."""
    url = f"https://www.apotheek.nl/medicijnen/{normalized_medicijn}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        ul_element = soup.find('ul', id='container')
        if ul_element:
            return ul_element.get_text(strip=True)
        else:
            return f"Geen details gevonden voor het medicijn {medicijn}."
    else:
        return f"Medicijn '{medicijn}' is not valid."
